In [3]:
!pip install kafka-python

In [5]:
topic = 'kafka-test'
kafka_server = 'localhost:9092'

In [6]:
from kafka import KafkaProducer

# Kafka Producer 생성
producer = KafkaProducer(
    bootstrap_servers=kafka_server,  # Kafka 서버 주소
    value_serializer=lambda v: v.encode('utf-8')  # 메시지 직렬화
)

# 토픽명


# 메시지 전송
for i in range(5):
    message = f"Message {i}"
    producer.send(topic, value=message)
    print(f"Sent: {message}")

# Producer 종료
producer.close()

Sent: Message 0
Sent: Message 1
Sent: Message 2
Sent: Message 3
Sent: Message 4


In [7]:
from kafka import KafkaConsumer

# Kafka Consumer 생성
consumer = KafkaConsumer(
    topic,  # 구독할 토픽 이름
    bootstrap_servers= kafka_server,  # Kafka 서버 주소
    group_id='test-group',  # 컨슈머 그룹 ID
    auto_offset_reset='earliest',  # 메시지를 처음부터 읽기
    enable_auto_commit=True,  # 자동 커밋 활성화
    value_deserializer=lambda x: x.decode('utf-8')  # 메시지 디코딩
)

print("Kafka Consumer 시작. 메시지 대기 중...")

try:
    for message in consumer:
        print(f"Received message: Key={message.key}, Value={message.value}")
except KeyboardInterrupt:
    print("Consumer 종료.")
finally:
    consumer.close()

Kafka Consumer 시작. 메시지 대기 중...
Received message: Key=None, Valueello
Received message: Key=None, Value=nice to meet you
Received message: Key=None, Value=hello kafka
Received message: Key=None, Value=Message 0
Received message: Key=None, Value=Message 1
Received message: Key=None, Value=Message 2
Received message: Key=None, Value=Message 3
Received message: Key=None, Value=Message 4
Consumer 종료.


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Kafka-Spark Integration") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/lab11/.ivy2/cache
The jars for the packages stored in: /home/lab11/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-94b61a7b-21c4-4f03-af66-46d5496b9769;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 648ms :: artifacts dl 23ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central i

In [2]:
kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "kafka-test") \
    .load()

kafka_data = kafka_stream.selectExpr("CAST(value AS STRING)")

query = kafka_data.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

print("Streaming is running...")
query.awaitTermination()

24/12/18 13:32:57 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-30b7a252-16f8-4039-ab06-65e3886ce708. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


Streaming is running...
-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------+
|             value|
+------------------+
|testmeassssage|
+------------------+



KeyboardInterrupt: 